# Análisis espacio temporal de la dinámica del delito en el territorio colombiano

Carga el conjunto de datos, limpia filas irrelevantes, ajusta formatos de fechas y textos, convierte los valores numéricos a enteros y elimina una columna innecesaria, todo para preparar los datos de manera más consistente y uniforme antes de analizarlos.

In [2]:
import pandas as pd

# Cargar el archivo CSV (ajusta el nombre y ruta del archivo)
df = pd.read_csv('Datos2010-2015.csv')

# Suponiendo que la columna se llama 'fecha'
# Convertir de string a datetime con formato original
df['FECHA_HECHO'] = pd.to_datetime(df['FECHA_HECHO'], format='%Y-%m-%d')

# Luego convertirlo al formato YYYY-MM-DD como string
df['FECHA_HECHO'] = df['FECHA_HECHO'].dt.strftime('%Y-%m-%d')

# Verifica los cambios
print(df.head())

# (Opcional) Guardar en un nuevo CSV
df.to_csv('Datos2010-2015.csv', index=False)


C:\Users\monte\AppData\Local\Temp\ipykernel_19716\706594315.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Datos2010-2015.csv')


  DEPARTAMENTO     MUNICIPIO CODIGO_DANE FECHA_HECHO     GENERO  \
0     AMAZONAS  LETICIA (CT)    91001000  2010-01-01  MASCULINO   
1     AMAZONAS  LETICIA (CT)    91001000  2010-01-01  MASCULINO   
2     AMAZONAS  LETICIA (CT)    91001000  2010-01-01  MASCULINO   
3     AMAZONAS  LETICIA (CT)    91001000  2010-01-02  MASCULINO   
4     AMAZONAS  LETICIA (CT)    91001000  2010-01-02  MASCULINO   

                        DESCRIPCION_CONDUCTA_CAPTURA GRUPO_ETARIO  CANTIDAD   
0    ARTÍCULO 429. VIOLENCIA CONTRA SERVIDOR PÚBLICO      ADULTOS          1  
1  ARTÍCULO 376. TRÁFICO. FABRICACIÓN O PORTE DE ...      ADULTOS          1  
2                  ARTÍCULO 111. LESIONES PERSONALES      ADULTOS          2  
3                    ARTÍCULO 239. HURTO RESIDENCIAS      ADULTOS          1  
4  ARTÍCULO 376. TRÁFICO. FABRICACIÓN O PORTE DE ...      ADULTOS          1  


In [3]:
import pandas as pd

# Load data into a DataFrame
general_data = pd.read_csv('Datos2010-2015.csv', low_memory=False)

# Remove spurious rows based on 'DEPARTAMENTO'
general_data = general_data[~general_data["DEPARTAMENTO"].isin(["QUITO", "MADRID", "NEW YORK", "NEW JERSEY", "FLORIDA"])]

# Convert 'FECHA_HECHO' column to datetime format
general_data['FECHA_HECHO'] = pd.to_datetime(general_data['FECHA_HECHO'], format="%Y-%m-%d")

# Convert all string columns to uppercase
str_cols = general_data.select_dtypes(include=['object']).columns
general_data[str_cols] = general_data[str_cols].apply(lambda x: x.str.upper())

# Select numeric columns
num_cols = general_data.select_dtypes(include=['float64', 'int64']).columns

# Convert numeric columns to integer type
general_data[num_cols] = general_data[num_cols].astype(int)

# Drop the 'CODIGO_DANE' column
general_data.drop(columns=["CODIGO_DANE"], inplace=True)


Función para reemplazar valores en una columna del conjunto de datos cuando su frecuencia es menor a un determinado percentil. La función toma el DataFrame original, calcula el percentil de las frecuencias de la columna indicada y reemplaza esos valores por un valor especificado, devolviendo una copia del DataFrame con las modificaciones.</br> **Es usado para evitar listar crimenes poco relevantes**

In [4]:
import numpy as np

def replace_by_percentile(data, column, percentile, replacement_value):
    # Calcular el percentil de las frecuencias
    frequencies = data[column].value_counts()
    threshold = np.percentile(frequencies, percentile)
    
    # Crear una copia del DataFrame para evitar modificar el original
    data_copy = data.copy()
    
    # Reemplazar los valores en las filas donde la frecuencia es menor al percentil dado
    data_copy.loc[data_copy[column].map(frequencies) < threshold, column] = replacement_value
    
    return data_copy

modified_data = replace_by_percentile(general_data, "DESCRIPCION_CONDUCTA_CAPTURA", 88, "OTROS")

Crea un nuevo conjunto de datos ajustado según la geolocalización. Primero, convierte a Bogotá en un departamento independiente cambiando su nombre en las columnas correspondientes y eliminando duplicados. Luego, renombra ciertos departamentos para que tengan sus nombres completos, como "La Guajira" y "Valle del Cauca".

In [5]:
#Creamos un nuevo datasets, con los datos ajustados a la geolocalizacion
clean_data = modified_data.copy()

# Primero convertimos bogota en un departamento independiente
bogota_dc = modified_data[modified_data["MUNICIPIO"] == "BOGOTÁ D.C."]
bogota_dc.loc[:, "DEPARTAMENTO"] = "BOGOTÁ, D.C."
bogota_dc.loc[:, "MUNICIPIO"] = "BOGOTÁ, D.C."
clean_data = pd.concat([clean_data, bogota_dc], ignore_index=True)
clean_data = clean_data[clean_data["MUNICIPIO"] != "BOGOTÁ D.C."]

#Renombramos guajira a la guajira
clean_data.loc[clean_data["DEPARTAMENTO"] == "GUAJIRA", "DEPARTAMENTO"] = "LA GUAJIRA"
#Renombramos valle a valle del cauca
clean_data.loc[clean_data["DEPARTAMENTO"] == "VALLE", "DEPARTAMENTO"] = "VALLE DEL CAUCA"

In [ ]:
print(clean_data)

Función que convierte nombres de municipios a sus formas oficiales o completas usando un diccionario de correspondencia. Si un nombre está en el diccionario, se reemplaza; si no, se mantiene igual. Luego, aplica esta función a la columna de municipios en el conjunto de datos clean_data.</br>**Cambia los nombres de los municipios, por sus nombres oficiales**

In [6]:
def convert_names(original_names):
    # Diccionario de correspondencia entre los nombres
    dictionary  = {
        'CALI': 'SANTIAGO DE CALI',
        'LÓPEZ': 'LÓPEZ DE MICAY',
        'EL PAUJIL': 'EL PAUJÍL',
        'MAGÜI': 'MAGÜÍ',
        'CARTAGENA': 'CARTAGENA DE INDIAS',
        'SOTARA': 'SOTARÁ PAISPAMBA',
        'GACHALA': 'GACHALÁ',
        'PURÍSIMA': 'PURÍSIMA DE LA CONCEPCIÓN',
        'ANCUYÁ': 'ANCUYA',
        'CERRO SAN ANTONIO': 'CERRO DE SAN ANTONIO',
        'MARIQUITA': 'SAN SEBASTIÁN DE MARIQUITA',
        'SANTAFÉ DE ANTIOQUIA': 'SANTA FÉ DE ANTIOQUIA',
        'ITAGUI': 'ITAGÜÍ',
        'CACHIRÁ': 'CÁCHIRA',
        'ANZA': 'ANZÁ',
        'CARMEN DEL DARIEN': 'CARMEN DEL DARIÉN',
        'CUASPUD': 'CUASPUD CARLOSAMA',
        'COLOSO': 'COLOSÓ',
        'SAN ANDRÉS SOTAVENTO': 'SAN ANDRÉS DE SOTAVENTO',
        'SAN JUAN DE RÍO SECO': 'SAN JUAN DE RIOSECO',
        'GÜICÁN': 'GÜICÁN DE LA SIERRA',
        'SABANAS DE SAN ANGEL': 'SABANAS DE SAN ÁNGEL',
        'CAQUEZA': 'CÁQUEZA',
        'MOMPÓS': 'SANTA CRUZ DE MOMPOX',
        'CHAMEZA': 'CHÁMEZA',
        'CARURU': 'CARURÚ',
        'BELÉN DE LOS ANDAQUIES': 'BELÉN DE LOS ANDAQUÍES',
        'CONSACA': 'CONSACÁ',
        'CÚCUTA': 'SAN JOSÉ DE CÚCUTA',
        'GAMBITA': 'GÁMBITA',
        'ENTRERRIOS': 'ENTRERRÍOS',
        'TORIBIO': 'TORIBÍO',
        'SAN ANDRES DE TUMACO': 'SAN ANDRÉS DE TUMACO',
        'RÍO IRO': 'RÍO IRÓ',
        'EL PIÑON': 'EL PIÑÓN',
        'SONSON': 'SONSÓN',
        'MACHETA': 'MACHETÁ',
        'CALARCA': 'CALARCÁ',
        'TURBANÁ': 'TURBANA',
        'UMBITA': 'ÚMBITA',
        'GUAYABAL DE SIQUIMA': 'GUAYABAL DE SÍQUIMA',
        'GAMEZA': 'GÁMEZA',
        'IQUIRA': 'ÍQUIRA',
        'PIENDAMÓ': 'PIENDAMÓ - TUNÍA',
        'ABREGO': 'ÁBREGO',
        'SAN VICENTE': 'SAN VICENTE FERRER',
        'VILLA DE SAN DIEGO DE UBATE': 'VILLA DE SAN DIEGO DE UBATÉ',
        'DON MATÍAS': 'DONMATÍAS',
        'FOMEQUE': 'FÓMEQUE'
    }

    # Convertir los nombres utilizando el diccionario, o mantener el nombre original si no se encuentra
    converted_names = [dictionary.get(name, name) for name in original_names]
    
    return converted_names


# Aplicar la función a la columna 'MUNICIPIO' del DataFrame clean_data
clean_data['MUNICIPIO'] = convert_names (clean_data['MUNICIPIO'])

Carga un archivo JSON con datos geográficos de Colombia, ajusta los nombres de las columnas y elimina una columna innecesaria. Luego, fusiona estos datos con el conjunto de datos clean_data, filtra los registros relacionados con un delito específico, agrupa los datos por mes, departamento, municipio y coordenadas geográficas, y calcula la suma de los valores. Finalmente, extrae el mes de la fecha y organiza las columnas relevantes para obtener un conjunto de datos mensual con información geográfica.

In [7]:
geodf = pd.read_json("GeoColombia.json") \
    .rename(columns={"Departamento": "DEPARTAMENTO", "Localidad": "MUNICIPIO"}) \
    .drop(columns=["Tipo"])

monthly_data = pd.merge(clean_data, geodf, on=["DEPARTAMENTO", "MUNICIPIO"], how="inner") \
    .query("DESCRIPCION_CONDUCTA_CAPTURA == 'ARTÍCULO 376. TRÁFICO. FABRICACIÓN O PORTE DE ESTUPEFACIENTES'") \
    .groupby([pd.Grouper(key='FECHA_HECHO', freq='ME'), 'DEPARTAMENTO', 'MUNICIPIO', 'LATITUD', 'LONGITUD']) \
    .sum() \
    .reset_index()

monthly_data['MES'] = monthly_data['FECHA_HECHO'].dt.to_period('M')
monthly_data = monthly_data[['MES', 'LATITUD', 'LONGITUD', 'CANTIDAD ']]

In [ ]:
print(monthly_data)

Copia los datos mensuales y luego crea una nueva columna llamada 'lat_long', que combina las coordenadas de latitud y longitud con un formato específico para cumplir con las reglas de MATLAB. Se reemplazan los puntos por 'p' y los signos negativos por 'n'. Después, pivota el DataFrame para organizar los datos de manera que cada combinación única de mes y coordenadas se convierta en una columna, con los valores de la columna 'CANTIDAD' sumados cuando es necesario y los valores faltantes llenados con ceros.

In [8]:
# Copiar los datos mensuales
df = monthly_data.copy()

# Crear la columna 'lat_long' con un prefijo y reemplazar el separador para cumplir con las reglas de MATLAB
df['lat_long'] = (
    'loc_' +
    df['LATITUD'].astype(str).str.replace('.', 'p').str.replace('-', 'n') + # Reemplaza '.' con 'p' y '-' con 'n'
    'x' +
    df['LONGITUD'].astype(str).str.replace('.', 'p').str.replace('-', 'n')
)

# Pivotar el DataFrame
df_pivot = df.pivot_table(index='MES', columns='lat_long', values='CANTIDAD ', aggfunc='sum', fill_value=0)
df_pivot.to_csv("resultado_pivot.csv")

In [ ]:
print(df.pivot)

Carga un archivo CSV con los datos previamente pivotados y luego deshace el pivot (convierte las columnas 'lat_long' de nuevo en filas). Después, separa la columna 'lat_long' en dos nuevas columnas: 'LATITUD' y 'LONGITUD'. Los valores de estas columnas se ajustan reemplazando caracteres que fueron usados para formatear las coordenadas (como 'n' por '-' y 'p' por '.') para restaurar los valores numéricos originales. También elimina el prefijo 'loc_' y convierte las columnas 'LATITUD' y 'LONGITUD' a tipo numérico. Finalmente, elimina la columna 'lat_long' que ya no es necesaria.

In [ ]:
# Despivotar el DataFrame (convertir las columnas 'lat_long' en filas)
df_pivot = pd.read_csv('resultado_pivot.csv')
# Despivotar el DataFrame
df_unpivoted = pd.melt(df_pivot, id_vars=['MES'], var_name='lat_long', value_name='CANTIDAD')

# Separar la columna 'lat_long' en 'LATITUD' y 'LONGITUD'
df_unpivoted[['LATITUD', 'LONGITUD']] = df_unpivoted['lat_long'].str.split('x', expand=True)

# Reemplazar 'n' con '-' y 'p' con '.' para recuperar los valores numéricos originales
df_unpivoted['LATITUD'] = df_unpivoted['LATITUD'].str.replace('n', '-', regex=False).str.replace('p', '.', regex=False)
df_unpivoted['LONGITUD'] = df_unpivoted['LONGITUD'].str.replace('n', '-', regex=False).str.replace('p', '.', regex=False)

# Eliminar el prefijo 'loc_' de las coordenadas
df_unpivoted['LATITUD'] = df_unpivoted['LATITUD'].str.replace('loc_', '', regex=False)
df_unpivoted['LONGITUD'] = df_unpivoted['LONGITUD'].str.replace('loc_', '', regex=False)

# Convertir las columnas LATITUD y LONGITUD a tipo numérico
df_unpivoted['LATITUD'] = pd.to_numeric(df_unpivoted['LATITUD'])
df_unpivoted['LONGITUD'] = pd.to_numeric(df_unpivoted['LONGITUD'])

# Eliminar la columna 'lat_long' ya que ahora tenemos LATITUD y LONGITUD
df_unpivoted.drop(columns=['lat_long'], inplace=True)
df_unpivoted['CANTIDAD'] = df_unpivoted['CANTIDAD'].round(0).astype(int)
df_unpivoted[df_unpivoted["CANTIDAD"] > 0].to_csv('resultado_unpivot_rounded.csv', index=False)

Define una función que toma un conjunto de datos y los filtra según una columna específica (en este caso, "DEPARTAMENTO"). Luego, para cada valor único en esa columna, crea un archivo CSV con los datos correspondientes. La función también asegura que la carpeta donde se guardarán los archivos exista, y si no, la crea automáticamente. Para cada archivo generado, imprime el nombre del archivo y la cantidad de filas que contiene, aunque esta opción de impresión puede ser desactivada si se desea. Finalmente, la función guarda los archivos CSV en la carpeta especificada.

In [ ]:
import os

# Función para guardar un archivo CSV por cada valor único en una columna
# data: DataFrame con los datos
# filter_column: nombre de la columna a utilizar para filtrar los datos
# output_folder: carpeta donde guardar los archivos CSV
# out: si es True, imprime un mensaje al guardar cada archivo CSV
def csv_by_filter(data, filter_column, output_folder, out = True):
    # Especifica la carpeta donde guardar los archivos CSV
    os.makedirs(output_folder, exist_ok=True)  # Crea la carpeta si no existe

    # Filtra y guarda un CSV para cada departamento
    for filter_column, loop_data in data.groupby(filter_column):
        # Genera el nombre del archivo para cada departamento
        output_file = os.path.join(output_folder, f"{filter_column}.csv")
        if (out):
            print(f"Guardando {output_file}... con {len(loop_data)} filas")
        # Guarda el DataFrame filtrado como CSV
        loop_data.to_csv(output_file, index=False)
    if (out):
        print("Archivos CSV generados en la carpeta:", output_folder)

# Filtra y guarda un CSV para cada departamento
csv_by_filter(modified_data, "DEPARTAMENTO", 'CSV por Departamento', out = False)

### Series de tiempo, graficas de correlacion, graficas de correlacion parcial y graficos de dipsersion por lag (retardo) para departamentos 

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.stattools import acf
import numpy as np

# Cargar datos
cundinamarca = pd.read_csv('CSV por Departamento/CUNDINAMARCA.csv')

# Convertir la columna de fechas a datetime para trabajar con series de tiempo
cundinamarca['FECHA_HECHO'] = pd.to_datetime(cundinamarca['FECHA_HECHO'])
cundinamarca = cundinamarca.set_index('FECHA_HECHO')  # Configura el índice del DataFrame como fecha

# Función para cambiar la escala de la serie de tiempo
def cambiar_escala(df, escala='D', columna_valores=None):
    """
    Cambia la escala de tiempo de la serie.
    Parámetros:
        df: DataFrame original con índice de fechas
        escala: Frecuencia deseada ('D' para diaria, 'W' para semanal, 'M' para mensual, '2M' para bimestral,
                'Q' para trimestral, '6M' para semestral, 'Y' para anual)
        columna_valores: Columna a agregar como serie. Si es None, se usa el conteo de registros por fecha.
    """
    if columna_valores:
        serie = df[columna_valores].resample(escala).sum()
    else:
        serie = df.resample(escala).size()  # Cuenta de eventos en cada intervalo de tiempo
    return serie

# Selecciona la escala de tiempo deseada
escala = 'ME'  # Cambia a 'D', 'W', '2M', 'Q', '6M', 'Y' para diferentes escalas (diario, semanal, etc.)
columna_valores = None  # Si tienes una columna específica para sumar, cambia aquí su nombre

# Cambia la escala de la serie de tiempo
serie = cambiar_escala(cundinamarca, escala=escala, columna_valores=columna_valores)

# Visualización de la serie de tiempo con la escala seleccionada
fig = px.line(serie, title=f"Serie de Tiempo con Escala {escala}")
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="Valores")
fig.show()

# Parámetros de autocorrelación
num_lags = 30  # Ajusta el número de rezagos que deseas visualizar en los gráficos de ACF y PACF

# Cálculo de ACF y PACF
acf_vals = acf(serie, nlags=num_lags, alpha=0.05)  # alpha para el intervalo de confianza
pacf_vals = pacf(serie, nlags=num_lags)
lags = list(range(num_lags + 1))

# Gráfico de Autocorrelación (ACF) con intervalos de confianza
fig_acf = go.Figure()

# Añadir la traza de ACF
fig_acf.add_trace(go.Bar(x=lags, y=acf_vals[0], name="ACF", marker_color="blue"))

# Calcular intervalos de confianza
conf_int = 1.96 / np.sqrt(len(serie))  # 95% de confianza

# Añadir líneas de intervalo de confianza
fig_acf.add_trace(go.Scatter(x=lags, y=acf_vals[0] + conf_int, mode='lines', name='Upper CI', line=dict(color='red', dash='dash')))
fig_acf.add_trace(go.Scatter(x=lags, y=acf_vals[0] - conf_int, mode='lines', name='Lower CI', line=dict(color='red', dash='dash')))

fig_acf.update_layout(title="Gráfico de Autocorrelación (ACF)",
                      xaxis_title="Rezagos (Lags)",
                      yaxis_title="Autocorrelación")
fig_acf.show()

# Gráfico de Autocorrelación Parcial (PACF)
fig_pacf = go.Figure()
fig_pacf.add_trace(go.Bar(x=lags, y=pacf_vals, name="PACF", marker_color="orange"))
fig_pacf.update_layout(title="Gráfico de Autocorrelación Parcial (PACF)",
                       xaxis_title="Rezagos (Lags)",
                       yaxis_title="Autocorrelación Parcial")
fig_pacf.show()

# Gráficos de dispersión para cada retraso (lag)
num_lags_scatter = 12  # Selecciona cuántos lags deseas graficar
for lag in range(1, num_lags_scatter + 1):
    # Crear serie desplazada
    serie_lagged = serie.shift(lag)
    
    # Crear la figura para cada lag
    fig = px.scatter(x=serie_lagged, y=serie, title=f'Gráfico de Dispersión para Lag {lag}')
    fig.update_xaxes(title_text=f'Valores Rezagados (Lag {lag})')
    fig.update_yaxes(title_text='Valores Actuales')
    fig.show()


## series de tiempo desglosadas por municipio

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from statsmodels.tsa.stattools import acf, pacf

# Cargar y preparar los datos
cundinamarca = pd.read_csv("CSV por Departamento/CAUCA.csv")
cundinamarca['FECHA_HECHO'] = pd.to_datetime(cundinamarca['FECHA_HECHO'])

# Función para cambiar la escala temporal de la serie
def cambiar_escala(df, escala='ME', columna_fecha='FECHA_HECHO', columna_valores='CANTIDAD'):
    df = df.set_index(columna_fecha)
    serie = df[columna_valores].resample(escala).sum()
    return serie

# Definir la escala de tiempo deseada y el número de lags
escala = 'ME'  # Escala temporal: 'D' para diario, 'W' para semanal, 'ME' para mensual, etc.
num_lags = 30  # Número de lags para ACF y PACF

# Lista de municipios
municipios = cundinamarca['MUNICIPIO'].unique()

# Gráfico de series de tiempo, ACF, PACF para cada municipio
for municipio in municipios:
    datos_municipio = cundinamarca[cundinamarca['MUNICIPIO'] == municipio]  # Filtrar datos por municipio
    serie_municipio = cambiar_escala(datos_municipio, escala=escala)  # Cambiar la escala temporal
    
    # Gráfico de la serie de tiempo
    fig_tiempo = px.line(serie_municipio, title=f"Serie de Tiempo - {municipio}", labels={"value": "Cantidad", "FECHA_HECHO": "Fecha"})
    fig_tiempo.update_layout(xaxis_title="Fecha", yaxis_title="Cantidad de Delitos")
    fig_tiempo.show()
    
    # Cálculo de ACF y PACF
    acf_vals = acf(serie_municipio, nlags=num_lags)
    pacf_vals = pacf(serie_municipio, nlags=num_lags)
    lags = list(range(num_lags + 1))

    # Gráfico de Autocorrelación (ACF) con intervalo de confianza
    fig_acf = go.Figure()
    fig_acf.add_trace(go.Bar(x=lags, y=acf_vals, name="ACF"))
    conf_interval_acf = 1.96 / (len(serie_municipio) ** 0.5)
    fig_acf.add_shape(type="line", x0=0, x1=num_lags, y0=conf_interval_acf, y1=conf_interval_acf, line=dict(color="red", dash="dash"))
    fig_acf.add_shape(type="line", x0=0, x1=num_lags, y0=-conf_interval_acf, y1=-conf_interval_acf, line=dict(color="red", dash="dash"))
    fig_acf.update_layout(title=f"Gráfico de Autocorrelación (ACF) - {municipio}",
                          xaxis_title="Rezagos (Lags)",
                          yaxis_title="Autocorrelación")
    fig_acf.show()

    # Gráfico de Autocorrelación Parcial (PACF)
    fig_pacf = go.Figure()
    fig_pacf.add_trace(go.Bar(x=lags, y=pacf_vals, name="PACF", marker_color="orange"))
    conf_interval_pacf = 1.96 / (len(serie_municipio) ** 0.5)
    fig_pacf.add_shape(type="line", x0=0, x1=num_lags, y0=conf_interval_pacf, y1=conf_interval_pacf, line=dict(color="red", dash="dash"))
    fig_pacf.add_shape(type="line", x0=0, x1=num_lags, y0=-conf_interval_pacf, y1=-conf_interval_pacf, line=dict(color="red", dash="dash"))
    fig_pacf.update_layout(title=f"Gráfico de Autocorrelación Parcial (PACF) - {municipio}",
                           xaxis_title="Rezagos (Lags)",
                           yaxis_title="Autocorrelación Parcial")
    fig_pacf.show()

    # Función opcional para gráficos de dispersión de lags (comentada)
    
    # Gráficos de dispersión para diferentes lags
    def graficar_lag_disperion(serie, max_lag):
        for lag in range(1, max_lag + 1):
            fig_lag = px.scatter(x=serie[:-lag], y=serie[lag:], title=f"Gráfico de Dispersión - Retardo {lag}",
                                 labels={"x": f"Valor en t", "y": f"Valor en t + {lag}"})
            fig_lag.update_layout(xaxis_title="Valor en t", yaxis_title=f"Valor en t + {lag}")
            fig_lag.show()
    
    # Llamada a la función para graficar dispersión de lags hasta el número deseado
    graficar_lag_disperion(serie_municipio.values, max_lag=1)  # Cambia max_lag según el número de lags deseado
    


# selecciona un departamento y un municipio en espescifico

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from statsmodels.tsa.stattools import acf, pacf

# Cargar y preparar los datos
data = pd.read_csv("CSV por Departamento/CAUCA.csv")
data['FECHA_HECHO'] = pd.to_datetime(data['FECHA_HECHO'])

# Variables de selección para el departamento y municipio
departamento = "CAUCA"
municipio_seleccionado = "ROSAS"

# Filtrar datos por el departamento y municipio deseados
data_filtrada = data[(data['DEPARTAMENTO'] == departamento) & (data['MUNICIPIO'] == municipio_seleccionado)]

# Función para cambiar la escala temporal de la serie
def cambiar_escala(df, escala='ME', columna_fecha='FECHA_HECHO', columna_valores='CANTIDAD'):
    df = df.set_index(columna_fecha)
    serie = df[columna_valores].resample(escala).sum()
    return serie

# Definir la escala de tiempo deseada y el número de lags
escala = 'ME'  # Escala temporal: 'D' para diario, 'W' para semanal, 'ME' para mensual, etc.
num_lags = 30  # Número de lags para ACF y PACF

# Cambiar la escala temporal para el municipio seleccionado
serie_municipio = cambiar_escala(data_filtrada, escala=escala)

# Gráfico de la serie de tiempo
fig_tiempo = px.line(serie_municipio, title=f"Serie de Tiempo - {municipio_seleccionado}", labels={"value": "Cantidad", "FECHA_HECHO": "Fecha"})
fig_tiempo.update_layout(xaxis_title="Fecha", yaxis_title="Cantidad de Delitos")
fig_tiempo.show()

# Cálculo de ACF y PACF
acf_vals = acf(serie_municipio, nlags=num_lags)
pacf_vals = pacf(serie_municipio, nlags=num_lags)
lags = list(range(num_lags + 1))

# Gráfico de Autocorrelación (ACF) con intervalo de confianza
fig_acf = go.Figure()
fig_acf.add_trace(go.Bar(x=lags, y=acf_vals, name="ACF"))
conf_interval_acf = 1.96 / (len(serie_municipio) ** 0.5)
fig_acf.add_shape(type="line", x0=0, x1=num_lags, y0=conf_interval_acf, y1=conf_interval_acf, line=dict(color="red", dash="dash"))
fig_acf.add_shape(type="line", x0=0, x1=num_lags, y0=-conf_interval_acf, y1=-conf_interval_acf, line=dict(color="red", dash="dash"))
fig_acf.update_layout(title=f"Gráfico de Autocorrelación (ACF) - {municipio_seleccionado}",
                      xaxis_title="Rezagos (Lags)",
                      yaxis_title="Autocorrelación")
fig_acf.show()

# Gráfico de Autocorrelación Parcial (PACF)
fig_pacf = go.Figure()
fig_pacf.add_trace(go.Bar(x=lags, y=pacf_vals, name="PACF", marker_color="orange"))
conf_interval_pacf = 1.96 / (len(serie_municipio) ** 0.5)
fig_pacf.add_shape(type="line", x0=0, x1=num_lags, y0=conf_interval_pacf, y1=conf_interval_pacf, line=dict(color="red", dash="dash"))
fig_pacf.add_shape(type="line", x0=0, x1=num_lags, y0=-conf_interval_pacf, y1=-conf_interval_pacf, line=dict(color="red", dash="dash"))
fig_pacf.update_layout(title=f"Gráfico de Autocorrelación Parcial (PACF) - {municipio_seleccionado}",
                       xaxis_title="Rezagos (Lags)",
                       yaxis_title="Autocorrelación Parcial")
fig_pacf.show()

# Función para gráficos de dispersión de lags
def graficar_lag_dispersion(serie, max_lag):
    for lag in range(1, max_lag + 1):
        fig_lag = px.scatter(x=serie[:-lag], y=serie[lag:], title=f"Gráfico de Dispersión - Retardo {lag}",
                             labels={"x": f"Valor en t", "y": f"Valor en t + {lag}"})
        fig_lag.update_layout(xaxis_title="Valor en t", yaxis_title=f"Valor en t + {lag}")
        fig_lag.show()

# Llamada a la función para graficar dispersión de lags hasta el número deseado
graficar_lag_dispersion(serie_municipio.values, max_lag=3)  # Ajusta max_lag según el número de lags deseado


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.stattools import acf, pacf
import numpy as np

# Cargar datos
cundinamarca = pd.read_csv('CSV por Departamento/CAUCA.csv')

# Convertir la columna de fechas a datetime para trabajar con series de tiempo
cundinamarca['FECHA_HECHO'] = pd.to_datetime(cundinamarca['FECHA_HECHO'])
cundinamarca = cundinamarca.set_index('FECHA_HECHO')  # Configura el índice del DataFrame como fecha

# Función para cambiar la escala de la serie de tiempo y desglosar por genero
def cambiar_escala_por_genero(df, escala='M', columna_valores='CANTIDAD', columna_genero='GENERO'):
    """
    Cambia la escala de tiempo de la serie y desglosa por género.
    Parámetros:
        df: DataFrame original con índice de fechas
        escala: Frecuencia deseada ('D' para diaria, 'W' para semanal, 'M' para mensual, 'Q' para trimestral, 'Y' para anual)
        columna_valores: Columna con valores numéricos
        columna_genero: Columna que indica el género (masculino/femenino)
    """
    # Agrupar por la frecuencia seleccionada y por género, sumando la cantidad de incidentes
    serie = df.groupby([pd.Grouper(freq=escala), columna_genero])[columna_valores].sum().unstack()
    return serie

# Seleccionar escala y columna de valores
escala = 'ME'  # Mensual
columna_valores = 'CANTIDAD'

# Cambia la escala de la serie de tiempo y desglosa por género
serie_por_genero = cambiar_escala_por_genero(cundinamarca, escala=escala, columna_valores=columna_valores)

# Visualización de la serie de tiempo con la escala seleccionada, desglosada por género
fig = px.line(serie_por_genero, title=f"Serie de Tiempo con Escala {escala} por Género")
fig.update_xaxes(title_text="Fecha")
fig.update_yaxes(title_text="Cantidad")
fig.show()

# Parámetros de autocorrelación
num_lags = 30

# Cálculo de ACF y PACF por género
for genero in serie_por_genero.columns:
    serie = serie_por_genero[genero].dropna()  # Eliminar valores nulos

    # Cálculo de ACF y PACF
    acf_vals, acf_confint = acf(serie, nlags=num_lags, alpha=0.05)
    pacf_vals = pacf(serie, nlags=num_lags)
    lags = list(range(num_lags + 1))

    # Gráfico de Autocorrelación (ACF) con intervalos de confianza
    fig_acf = go.Figure()
    fig_acf.add_trace(go.Bar(x=lags, y=acf_vals, name="ACF", marker_color="blue"))

    for i in range(len(lags)):
        fig_acf.add_shape(type="line", x0=lags[i], x1=lags[i], y0=acf_confint[i, 0], y1=acf_confint[i, 1],
                          line=dict(color="red", dash="dash"))

    fig_acf.update_layout(title=f"ACF - {genero}",
                          xaxis_title="Rezagos (Lags)",
                          yaxis_title="Autocorrelación")
    fig_acf.show()

    # Gráfico de Autocorrelación Parcial (PACF)
    fig_pacf = go.Figure()
    fig_pacf.add_trace(go.Bar(x=lags, y=pacf_vals, name="PACF", marker_color="orange"))
    fig_pacf.update_layout(title=f"PACF - {genero}",
                           xaxis_title="Rezagos (Lags)",
                           yaxis_title="Autocorrelación Parcial")
    fig_pacf.show()
    
    # Gráfico de dispersión para los lags seleccionados
    num_lags_scatter = 12
    for lag in range(1, num_lags_scatter + 1):
        serie_lagged = serie.shift(lag)
        fig = px.scatter(x=serie_lagged, y=serie, title=f'Dispersión Lag {lag} - {genero}')
        fig.update_xaxes(title_text=f'Valores Rezagados (Lag {lag})')
        fig.update_yaxes(title_text='Valores Actuales')
        fig.show()


In [ ]:
import matplotlib.pyplot as plt

# Datos
meses = [
    "Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
    "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"
]
factores_estacionales = [
    0.9052, 1.0289, 1.0187, 1.0486, 1.0436, 0.5776,
    0.8616, 1.2293, 1.3710, 0.9078, 0.9841, 0.9670
]

# Paleta de colores personalizada
colores = ["#ff9999", "#66b3ff", "#99ff99", "#ffcc99", "#c2c2f0", "#ffb3e6",
           "#c4e17f", "#76d7c4", "#f7b7a3", "#f4a261", "#e9c46a", "#2a9d8f"]

# Crear gráfico de dona
plt.figure(figsize=(8, 6))
plt.pie(factores_estacionales, labels=meses, colors=colores, autopct='%1.1f%%', startangle=90, pctdistance=0.85)

# Crear el círculo interno para hacer el gráfico de dona
centro = plt.Circle((0, 0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centro)

# Título
plt.title('Factor Estacional por Mes')

# Guardar el gráfico como PNG
plt.savefig("grafico_dona.png", format="png", dpi=300)

# Mostrar gráfico
plt.show()
